In [1]:
%matplotlib inline
%config InlineBackend.figure_formats = ['png']
# %config InlineBackend.figure_formats = ['svg']

In [2]:
from datetime import time, timedelta, datetime, date
from pathlib import Path

import math
import itertools
import typing as T

import pyodbc
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

plt.rcParams['figure.figsize'] = [10, 5]

# IDADE >=60 then é idoso, else é jovem
# PERÍODOS 
#      Matutino 6h-8h59
#      Diurno 9h-20h59
#      Vespertino 21h-00h59
#      Noturno 1h-05h59

matutino = (time(6,0), time(9,0))
diurno = (time(9,0), time(21,0))
vespertino = (time(21,0), time(23,0))
noturno = (time(23,0), time(6,0))
limite_idoso = 60


In [3]:
# list(mapas.columns)    

In [4]:
#  Processo de importação dos dados a partir de uma base SQL
#  

bases_de_dados = {
    "dados_originais": r"C:\Users\marta\OneDrive\Desktop\DynaMAPA.mdb",
#    "dados_mapas_2021": r"C:\Users\marta\projeto_PA\mapa2021\DynaMAPA.mdb",
#    "cardiomapa": r"D:\Biocardios Dataset\mapa2021\CARDIOMAPA.md1"   
}

#access_path = Path(r"C:\Users\marta\OneDrive\Desktop\DynaMAPA.mdb")
#access_path = Path(r"C:\Users\marta\projeto_PA\mapa2021\DynaMAPA.mdb")
#access_path = Path(r"D:\Biocardios Dataset\mapa2021\CARDIOMAPA.md1")

In [5]:
#  Essas linhas se conectam ao MSAccess como um banco de dados, e puxam
#  os dados brutos a partir de 3 tabelas relacionadas.
#  
#  
#  Os dados são desnormalizados (a informação de nome, gênero, etc.. é replicada em todas as linhas), de 
#  forma que tenhamos todos os dados em um único dataframe inicial para análise.

# Python envia o comando SQL para o BD Access e recebe todos os resultados (fetchall)
def executa_sql(conexao, comando_sql):
    return conexao.execute(comando_sql).fetchall()
    

comando_sql = """SELECT Patients.PatientID, Patients.PatientName,  Patients.Birthday, Patients.Sex, 
                  Patients.Weight, Patients.Height, Items.DateItem,
                  Items.Systolic, Items.Diastolic, Exams.ExamDate
FROM ExamsItems as Items, Exams as Exams, Patients as Patients 
WHERE Items.ExamID=Exams.ExamID AND  Exams.PatientID=Patients.PatientID  and Patients.PatientID<>1
"""

def le_base_de_dados(access_path):
    conexao = pyodbc.connect(f'Driver={{Microsoft Access Driver (*.mdb, *.accdb)}};DBQ={Path(access_path)};')
    mapas_dos_campos = pd.DataFrame([[item for item in row] for row in executa_sql(conexao, comando_sql)], 
            columns="numero_identificacao nome data_nascimento genero peso altura dateitem systolic diastolic data_exame".split() )

    mapas_dos_campos.systolic.loc[mapas_dos_campos.systolic == 0] = float("NaN")
    mapas_dos_campos.diastolic.loc[mapas_dos_campos.diastolic == 0] = float("NaN")
    return mapas_dos_campos

def le_todas_as_bases():
    dados = []
    for base in bases_de_dados.values():
        dados.append(le_base_de_dados(base))
    mapas_dos_campos = pd.concat(dados, axis=0, ignore_index=True)
    return mapas_dos_campos

mapas_dos_campos = le_todas_as_bases()
mapas_dos_campos

c:\users\marta\anaconda3\envs\emoti\lib\site-packages\pandas\core\indexing.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


Wall time: 19.7 s


,numero_identificacao,nome,data_nascimento,genero,peso,altura,dateitem,systolic,diastolic,data_exame
0,4,DONIZZETTE PEREIRA DA,1923-10-05,M,85,160,2018-09-10 12:39:00,140.0,80.0,2018-09-10 12:25:23
1,4,DONIZZETTE PEREIRA DA,1923-10-05,M,85,160,2018-09-10 12:48:00,NaN,NaN,2018-09-10 12:25:23
2,4,DONIZZETTE PEREIRA DA,1923-10-05,M,85,160,2018-09-10 13:03:00,NaN,NaN,2018-09-10 12:25:23
3,4,DONIZZETTE PEREIRA DA,1923-10-05,M,85,160,2018-09-10 13:18:00,NaN,NaN,2018-09-10 12:25:23
4,4,DONIZZETTE PEREIRA DA,1923-10-05,M,85,160,2018-09-10 13:33:00,NaN,NaN,2018-09-10 12:25:23
...,...,...,...,...,...,...,...,...,...,...
1655823,21793,FERNANDO LUIZ DE AZEREDO COUTINH,1971-01-14,M,99,179,2020-09-11 14:30:00,131.0,91.0,2020-09-10 17:42:01
1655824,21793,FERNANDO LUIZ DE AZEREDO COUTINH,1971-01-14,M,99,179,2020-09-11 14:45:00,143.0,84.0,2020-09-10 17:42:01
1655825,21793,FERNANDO LUIZ DE AZEREDO COUTINH,1971-01-14,M,99,179,2020-09-11 15:00:00,138.0,99.0,2020-09-10 17:42:01
1655826,21793,FERNANDO LUIZ DE AZEREDO COUTINH,1971-01-14,M,99,179,2020-09-11 15:15:00,127.0,89.0,2020-09-10 17:42:01


In [6]:
# Agrupa dados de cada paciente (que estão dispersos em uma linha por medida)

mapas_por_paciente = mapas_dos_campos.groupby("numero_identificacao")

def cria_mapas(df):
    """Monta as informações de cada paciente como uma linha do dataframe, na ordem que será usada daqui pra frente"""
    results = []
    for i, paciente in df:
        linha = [paciente.numero_identificacao.iloc[0], paciente.nome.iloc[0],
                 paciente.data_nascimento.iloc[0], paciente.genero.iloc[0],
                 paciente.peso.iloc[0], paciente.altura.iloc[0],
                 paciente.data_exame.iloc[0],
                ]
        
        # Cálculo da idade
        idade = (linha[-1] - linha[2]).days // 365.25

        # Se faltou uma das datas e não foi possível calcular a idade, marca a mesma como NaN - senão 
        # transforma em inteiro:
        if math.isnan(idade):
            print(linha, idade)
            idade = float("NaN")
        else:
            idade = int(idade)
        
        linha.append(idade)
        results.append(linha)
    results = pd.DataFrame(results, columns="numero_identificacao nome data_nascimento genero peso altura data_exame idade".split())
    return results

mapas_agrupado = cria_mapas(mapas_por_paciente)
mapas_agrupado

[1128, 'TESTE', NaT, '', '', '', Timestamp('2019-02-06 09:31:58')] nan
[3957, 'LUIZ GOMES DA ', NaT, '', '80', '170', Timestamp('2019-04-01 16:28:27')] nan
[6602, 'RITA TEIXEIRA DOS REIS ', NaT, '', '', '', Timestamp('2019-06-04 11:19:21')] nan
[6687, 'MARIA DO CARMO DA CONCEICAO ', NaT, '', '', '', Timestamp('2019-06-06 08:01:47')] nan
[7869, 'JOSE ANTONIO RIOS DA ', NaT, '', '', '', Timestamp('2019-07-05 08:38:10')] nan
[11210, 'JOAO RIBEIRO SIRQUEIRA ', NaT, '', '', '', Timestamp('2019-09-26 09:54:19')] nan
[11850, 'REGINA AUXILIADORA PEREIRA ', NaT, '', '', '', Timestamp('2019-10-15 08:01:02')] nan
[12212, 'DANIEL LOPES DA CUNHA TORRES DOS', NaT, '', '', '', Timestamp('2019-10-22 11:41:43')] nan
[13397, 'TESTE ', NaT, '', '', '', Timestamp('2019-11-23 09:21:23')] nan
[13398, 'TESTE 2C', NaT, '', '', '', Timestamp('2019-11-23 09:25:20')] nan
[14941, 'REGINA CARVALHO DE ', NaT, 'F', '80', '167', Timestamp('2020-01-16 14:29:32')] nan
[15273, 'ANA CRISTHINA GONCALVES DE ', NaT, '', '',

,numero_identificacao,nome,data_nascimento,genero,peso,altura,data_exame,idade
0,4,DONIZZETTE PEREIRA DA,1923-10-05,M,85,160,2018-09-10 12:25:23,94.0
1,5,JEFFERSON LUCENA,1948-10-07,M,80,165,2018-12-26 07:39:41,70.0
2,6,IGOR DAMASCENO,1999-03-23,M,100,197,2018-12-26 08:06:19,19.0
3,7,ANDREIA DE OLIVEIRA,1978-01-26,F,65,"1,64",2018-12-26 08:19:06,40.0
4,8,CRISTIANO NASCIMENTO POMBO DA,1990-03-07,M,120,172,2018-12-26 08:26:59,28.0
...,...,...,...,...,...,...,...,...
21132,21787,CASTRUZ CATRAMBY,1970-10-19,M,90,183,2020-09-10 16:21:02,49.0
21133,21788,TATIANE ROCHA DA,1985-03-15,F,93,167,2020-09-10 16:44:53,35.0
21134,21791,MARCOS PAULO,1973-10-31,M,81,170,2020-09-10 17:26:00,46.0
21135,21792,EULINA SOUZA,1952-04-21,F,64,160,2020-09-10 17:25:50,68.0


In [7]:
# Celula apenas para visualização dos dados intermediários
##################################################################################################
#pessoa = mapas_por_paciente.get_group(15626)
#pessoa.dateitem[0].date()
#pessoa
# pessoa[pessoa.dateitem > datetime(2018,9, 10, 18,0)].loc[:, "systolic"]



In [8]:
# Exemplo de operação com objetos do tipo "date" e "datetime"

a = datetime(2021, 9,30, 15, 0)
b = date(2020, 10, 1)
(a.date() - b).days

364

In [10]:
#    Normaliza o horário de medidas para contagem a partir de um horario do primeiro dia
#    (ex. a partir das 9h00).
#    
#    ts_col (string): nome da coluna com os dados de timestamp
#    starting_time (datetime.time): [9h00] horario a usar de base para normalizar as medidas 
#    name (str): Name of the resulting series
#    
#    Se o total do periodo de medidas ultrapassar o horário de meia-noite, após 24 horas as medidas
#    darão "overlap" sendo novamente categorizadas no dia 1.
#    
#    O retorno é um pd.Series com os tempos recalculados, ou a partir da 
#    dada-base de 2010-01-01.
#
#    usar o resultado com pd.concat para anexar a coluna de horario ao dataframe original.
 
def normalize_date(df, ts_col, starting_time=time(9,0), name="normalized_ts")->pd.Series:
    
    base = datetime(2010, 1,1, starting_time.hour, starting_time.minute)
    convert = lambda ts: datetime(base.year, base.month, base.day, ts.hour, ts.minute, ts.second)
    date_change_offset = lambda ts: timedelta(seconds=3600 * 24 * int(ts.time() < starting_time)) 
    result =  df.loc[:, ts_col].apply(lambda ts: (base + (convert(ts) - base) + date_change_offset(ts))) 
    result.name = name
    return result

    
#list(normalize_date(pessoa, "dateitem"))
#pessoa

In [11]:
def horario_noturno(hora):
    return (hora >= noturno[0].hour or hora < noturno[1].hour)
   
    
def extract_whitecoat(paciente):

#    Devolve uma sequencia com 4 tuplas de comprimento 3 (horário, medida sistólica, medida diastólica),
#    correspondentes as 4 primeiras medidas tomadas em cada exame.
#    
#    esses 12 valores são guardados em uma coluna a parte no dataframe de saída,
#    e são usados para detectar o padrão 'whitecoat' num momento posterior
    

    try:
        primeira_medida = paciente.dateitem.values[0]
        quarta_medida = paciente.dateitem.values[3]
    except IndexError: 
        # Paciente não tem as 4 primeiras medidas - deixamos as medidas de whitecoat em branco
        print("paciente sem dados das primeiras medidas:", paciente)
        return f"[{','.join('(nan, nan, nan)' for _ in range(4))} ]"
        
    primeiras_medidas = (paciente[paciente.dateitem >= primeira_medida][paciente.dateitem <= quarta_medida]
              [["dateitem", "systolic", "diastolic"]])
    whitecoat = [tuple(x) for x in primeiras_medidas.to_numpy()]
    whitecoat = str([(ts.strftime("%H:%M") , sist, diast)  for ts, sist, diast in whitecoat])
    return whitecoat
    
    
def transpoe_medidas(dados, limit=None):

#    Transforma as medições na vertical em uma única linha de dados,
#    com as medições em um período de 24h para cada paciente.
#    
#    Recebe um dataframe já com  um "group_by" por paciente, gerado pela 
#    função "mapas_dos_campos" acima.
#    
#    Cria colunas por horario, com intervalos de 15 minutos nos períodos matutino (> 6h00am), diurno e vespertino,
#    e intervalos de 30 min no período noturno (23h00 <= horário < 6h00)
#
#    Para achar a medida correspondente a cada horário, ele procura nos dados a medida
#    no horário mais alto dentro de um intervalo iniciando no horário da coluna:
#    isso é, para a coluna das 17h00, em que o intervalo entre medidas é
#    de 15 minutos, são olhadas as medidas das 17h00 até 17h14h59
#    Se não houver nenhuma medida nesse horário, a coluna
#    das 17h00 é marcada como sem dados (NaN). Se houver mais de uma, é usada a última.
    

    result = []
    cols = {}
    hora_base = datetime(2010, 1, 1, 9, 0)
    intervalo = 15
    
    
    for i in range(0, 24 * 60 // intervalo):
        td = timedelta(minutes=i * intervalo)
        dia = 0 if (td + timedelta(hours=9)  ).days < 1 else 1
        horas = int(td.total_seconds() // 3600 + 9) % 24
        minutos = int(td.total_seconds() % 3600) //60
        
        # Não cria intervalos de "15 min" se o horario for no período noturno.
        # (a medida padrão do exame passa para uma leitura a cada 30min nesse horário)
        if minutos in (15, 45) and horario_noturno(horas):
            continue
        
        cols[td] = f"+{dia} {horas}:{minutos:02d}"

    if limit:
        limiter = range(limit)
    else:
        limiter = itertools.count()
   
    for _, (paciente_id, paciente) in zip(limiter, dados):
        
        data_inicial = paciente.dateitem.iloc[0].date()
        
        paciente_norm = pd.concat((paciente, normalize_date(paciente, "dateitem", name="normalized_ts") ), axis=1)
        
        whitecoat = extract_whitecoat(paciente_norm)
        
        # Imprime os dados brutos de um paciente a partir de um certo horário
        # pode ser usado para verificar se a funcionalidade de normalizar o horáio abaixo está pegando as medias corretas
        # print(paciente_norm[paciente_norm["normalized_ts"] > datetime(2010, 1, 2) ])
        
        
        linha_sist = []
        linha_diast = []
        
        erro_diurno = timedelta(minutes=15)
        erro_noturno = timedelta(minutes=30)
        
        for col in cols:
            hora_normalizada = hora_base + col
            erro = erro_noturno if horario_noturno(hora_normalizada.hour) else erro_diurno
            loc_medida = paciente_norm[hora_normalizada <= paciente_norm.normalized_ts][paciente_norm.normalized_ts < hora_normalizada + erro]
            medida_sist = loc_medida.loc[:, "systolic"]
            medida_diast = loc_medida.loc[:, "diastolic"]
               
            if not len(medida_sist):
                medida_sist = medida_diast = float("nan")
                
            else:
                medida_sist = medida_sist.values[-1]
                medida_diast = medida_diast.values[-1]
        
            linha_sist.append(medida_sist)
            linha_diast.append(medida_diast)
            # linha.append(paciente_norm[inicio_jan <= paciente_norm.normalized_ts][paciente_norm.normalized_ts < final_jan].loc[:, coluna].mean())
            
        result.append(linha_sist + linha_diast + [whitecoat])
      
    colunas_sist = [f"sist {col.split()[-1]}" for col in cols.values()]
    colunas_diast = [f"diast {col.split()[-1]}" for col in cols.values()]
    coluna_whitecoat = ["whitecoat"]
    result = pd.DataFrame(result , columns=colunas_sist + colunas_diast + coluna_whitecoat)
    
    return result


In [44]:
lista_campos = ['numero_identificacao',
                 'nome',
                 'data_nascimento',
                 'genero',
                 'dateitem',
                 'systolic',
                 'diastolic',
                 'data_exame']

mapas_dos_campos_hora_original = mapas_dos_campos[lista_campos]
mapas_dos_campos_hora_original['dateitem'] = mapas_dos_campos_hora_original['dateitem'].apply(lambda elemento: elemento.strftime('%H:%M:%S'))

mapas_dos_campos_hora_original.to_csv(r"D:\Projeto_PA\dataset_modificado\nomes\pacientes_sem_carrossel.csv", 
                   sep=";", encoding="latin1", index=False)

mapas_dos_campos_hora_original

c:\users\marta\anaconda3\envs\emoti\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()


,numero_identificacao,nome,data_nascimento,genero,dateitem,systolic,diastolic,data_exame
0,4,DONIZZETTE PEREIRA DA,1923-10-05,M,12:39:00,140.0,80.0,2018-09-10 12:25:23
1,4,DONIZZETTE PEREIRA DA,1923-10-05,M,12:48:00,NaN,NaN,2018-09-10 12:25:23
2,4,DONIZZETTE PEREIRA DA,1923-10-05,M,13:03:00,NaN,NaN,2018-09-10 12:25:23
3,4,DONIZZETTE PEREIRA DA,1923-10-05,M,13:18:00,NaN,NaN,2018-09-10 12:25:23
4,4,DONIZZETTE PEREIRA DA,1923-10-05,M,13:33:00,NaN,NaN,2018-09-10 12:25:23
...,...,...,...,...,...,...,...,...
1655823,21793,FERNANDO LUIZ DE AZEREDO COUTINH,1971-01-14,M,14:30:00,131.0,91.0,2020-09-10 17:42:01
1655824,21793,FERNANDO LUIZ DE AZEREDO COUTINH,1971-01-14,M,14:45:00,143.0,84.0,2020-09-10 17:42:01
1655825,21793,FERNANDO LUIZ DE AZEREDO COUTINH,1971-01-14,M,15:00:00,138.0,99.0,2020-09-10 17:42:01
1655826,21793,FERNANDO LUIZ DE AZEREDO COUTINH,1971-01-14,M,15:15:00,127.0,89.0,2020-09-10 17:42:01


In [12]:
%time dados_transpostos = transpoe_medidas(mapas_por_paciente)

# dados_transpostos

c:\users\marta\anaconda3\envs\emoti\lib\site-packages\ipykernel_launcher.py:94: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
c:\users\marta\anaconda3\envs\emoti\lib\site-packages\ipykernel_launcher.py:22: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


paciente sem dados das primeiras medidas:         numero_identificacao   nome data_nascimento genero peso altura  \
359639                  1128  TESTE             NaT                      
359640                  1128  TESTE             NaT                      
359641                  1128  TESTE             NaT                      

                  dateitem  systolic  diastolic          data_exame  \
359639 2019-02-06 08:47:00     104.0       55.0 2019-02-06 09:31:58   
359640 2019-02-06 08:52:00       NaN        NaN 2019-02-06 09:31:58   
359641 2019-02-06 08:53:00     101.0       60.0 2019-02-06 09:31:58   

             normalized_ts  
359639 2010-01-02 08:47:00  
359640 2010-01-02 08:52:00  
359641 2010-01-02 08:53:00  
paciente sem dados das primeiras medidas:         numero_identificacao              nome data_nascimento genero peso  \
456319                  5961  ROMILDO DA SILVA      1977-04-28      M   59   
456320                  5961  ROMILDO DA SILVA      1977-04-28

In [13]:
dados_transpostos

,sist 9:00,sist 9:15,sist 9:30,sist 9:45,sist 10:00,sist 10:15,sist 10:30,sist 10:45,sist 11:00,sist 11:15,...,diast 6:45,diast 7:00,diast 7:15,diast 7:30,diast 7:45,diast 8:00,diast 8:15,diast 8:30,diast 8:45,whitecoat
0,105.0,108.0,116.0,104.0,106.0,98.0,NaN,107.0,NaN,NaN,...,NaN,NaN,NaN,NaN,67.0,61.0,69.0,62.0,51.0,"[('12:39', 140.0, 80.0), ('12:48', nan, nan), ..."
1,103.0,114.0,82.0,104.0,98.0,106.0,108.0,107.0,109.0,100.0,...,NaN,74.0,NaN,NaN,NaN,60.0,74.0,70.0,67.0,"[('08:04', 114.0, 60.0), ('08:15', 111.0, 74.0..."
2,147.0,148.0,156.0,140.0,130.0,132.0,121.0,113.0,149.0,128.0,...,NaN,93.0,NaN,NaN,NaN,86.0,87.0,82.0,82.0,"[('08:11', 133.0, 86.0), ('08:15', 137.0, 87.0..."
3,140.0,139.0,142.0,137.0,135.0,177.0,148.0,152.0,135.0,136.0,...,NaN,62.0,NaN,104.0,NaN,NaN,89.0,90.0,72.0,"[('08:22', 153.0, 89.0), ('08:30', 153.0, 90.0..."
4,NaN,110.0,101.0,108.0,111.0,111.0,114.0,115.0,113.0,112.0,...,NaN,NaN,93.0,68.0,91.0,NaN,NaN,81.0,72.0,"[('08:31', 106.0, 81.0), ('08:48', 116.0, 72.0..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21132,114.0,120.0,164.0,116.0,119.0,126.0,123.0,NaN,122.0,159.0,...,NaN,59.0,74.0,68.0,86.0,NaN,107.0,106.0,93.0,"[('16:29', 115.0, 72.0), ('16:48', nan, nan), ..."
21133,107.0,110.0,103.0,NaN,NaN,NaN,114.0,144.0,128.0,104.0,...,NaN,50.0,53.0,52.0,62.0,68.0,61.0,64.0,60.0,"[('16:51', 126.0, 68.0), ('17:00', 128.0, 60.0..."
21134,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,68.0,83.0,84.0,85.0,NaN,NaN,NaN,NaN,NaN,"[('17:26', 106.0, 75.0), ('17:41', 112.0, 85.0..."
21135,94.0,107.0,112.0,124.0,120.0,94.0,102.0,113.0,104.0,96.0,...,NaN,64.0,76.0,NaN,69.0,75.0,74.0,69.0,67.0,"[('17:54', 132.0, 77.0), ('18:03', nan, nan), ..."


In [14]:
# Cria colunas de cabeçalho para cada exame, a partir do "mapa agrupado"
# 

def cria_cabecalho(mapas_agrupado):
    
    mapas = mapas_agrupado[["numero_identificacao", "data_nascimento", "genero", "idade", "peso", "altura", "data_exame"] ]
    #mapas_nome = mapas_nome_agrupado[["numero_identificacao", "nome", "data_nascimento", "genero", "idade", "peso", "altura", "data_exame"] ]
    #print(mapas)
    return mapas

mapas = cria_cabecalho(mapas_agrupado)



def cria_nome_cabecalho(mapas_nome_agrupado):
 
    mapas_nome = mapas_agrupado[["numero_identificacao", "nome", "data_nascimento", "genero", "idade", "peso", "altura", "data_exame"] ]
    print(mapas_nome)
    return mapas_nome

mapas_nome = cria_nome_cabecalho(mapas_agrupado)




       numero_identificacao                              nome data_nascimento  \
0                         4            DONIZZETTE PEREIRA DA       1923-10-05   
1                         5                 JEFFERSON LUCENA       1948-10-07   
2                         6                    IGOR DAMASCENO      1999-03-23   
3                         7              ANDREIA DE OLIVEIRA       1978-01-26   
4                         8     CRISTIANO NASCIMENTO POMBO DA      1990-03-07   
...                     ...                               ...             ...   
21132                 21787                 CASTRUZ CATRAMBY       1970-10-19   
21133                 21788                 TATIANE ROCHA DA       1985-03-15   
21134                 21791                     MARCOS PAULO       1973-10-31   
21135                 21792                     EULINA SOUZA       1952-04-21   
21136                 21793  FERNANDO LUIZ DE AZEREDO COUTINH      1971-01-14   

      genero  idade peso al

In [15]:
mapas_final = pd.concat([mapas.iloc[:, :], dados_transpostos],axis=1)
mapas_nome_final = pd.concat([mapas_nome.iloc[:, :], dados_transpostos],axis=1)
mapas_nome

#######################################################################################################################
mapas_final.to_csv(r"D:\Projeto_PA\dataset_modificado\pacientes_com_medidas_whitecoat.csv", 
                   sep=";", encoding="latin1", index=False)

mapas_nome_final.to_csv(r"D:\Projeto_PA\dataset_modificado\nomes\pacientes_com_medidas_NOMES.csv", 
                   sep=";", encoding="latin1", index=False)

#mapas_final.to_csv(r"D:\Projeto_PA\dataset_modificado\pacientes_com_medidas_whitecoat_MAPA2021.csv", 
#                   sep=";", encoding="latin1", index=False)

In [16]:
mapas_final[mapas_final['numero_identificacao'] == 13659]

,numero_identificacao,data_nascimento,genero,idade,peso,altura,data_exame,sist 9:00,sist 9:15,sist 9:30,...,diast 6:45,diast 7:00,diast 7:15,diast 7:30,diast 7:45,diast 8:00,diast 8:15,diast 8:30,diast 8:45,whitecoat
13287,13659,1978-08-30,F,41.0,86,"1,72",2019-12-02 09:09:58,NaN,113.0,NaN,...,NaN,76.0,81.0,79.0,80.0,66.0,78.0,NaN,NaN,[]
